In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [3]:
from proj1_helpers import *
DATA_TRAIN_PATH = '/Users/alperkose/Desktop/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [ ]:
tX[tX==-999]=0
matr=np.zeros((2,2))
amq=np.zeros((30))
for ay in range (30):
    matr=np.corrcoef(y,tX[:,ay])
    amq[ay]=matr[0,1]
    if abs(amq[ay])>0.1:
        print(ay)
#amq.sort()
#amq

In [ ]:
tX[tX==-999]=0
np.corrcoef(y,tX[:,28])

In [ ]:
def split_data(x, y, ratio,seed=1):
    """split the dataset based on the split ratio."""
    # set seed
    np.random.seed(seed)
    # ***************************************************
    # INSERT YOUR CODE HERE
    # split the data based on the given ratio: TODO
    
    
    c = list(zip(x, y))

    np.random.shuffle(c)

    x, y = zip(*c)
    
    ind=round(ratio*len(x))
    ind2=len(x)
    x=np.array(x)
    trainx = x[0:ind-1,:]
    trainy = y[0:ind-1]
    testx = x[ind:ind2-1,:]
    testy = y[ind:ind2-1]
    
    return trainx, testx,trainy,testy
    # ***************************************************
    raise NotImplementedError

In [ ]:
tX,testx,y,testy=split_data(tX, y, 0.8,seed=1)

In [ ]:
tX=np.delete(tX,[14,15,17,18,20,24,25,27,28],1)
tX[tX==-999]=0
m=np.mean(tX,axis=0)
centered_tX=tX-m
centered_tX[centered_tX==-m]=0


standardized_tX=centered_tX / np.std(centered_tX, axis=0)

d=len(standardized_tX[0])
n=len(standardized_tX)
mat=np.zeros((n,1771+int(d*(d+1)/2+d)))
q=0
tao=d 

for i in range (n):
    for t in range (d):  
        mat[i,t]=standardized_tX[i,t]

        
for i in range (n):
    for monsoon in range (d):
        for t in range (monsoon,d): 
            mat[i,tao]=standardized_tX[i,t]*standardized_tX[i,monsoon]
            tao=tao+1
    tao=d  
    
    

tao=int(d*(d+1)/2+d)
    
for i in range (n):
    for r in range (d):
        for monsoon in range (r,d):
            for t in range (monsoon,d): 
                mat[i,tao]=standardized_tX[i,t]*standardized_tX[i,monsoon]*standardized_tX[i,r]
                tao=tao+1
    tao=int(d*(d+1)/2+d)  
        
    



In [ ]:
num_samples=len(mat)
tx = np.c_[np.ones(num_samples), mat]

In [ ]:
def least_squares(y, tx):
    """calculate the least squares solution."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # least squares: TODO
    # returns mse, and optimal weights
    xtx=np.dot(tx.transpose(),tx)
    xy=np.dot(tx.transpose(),y)
    w=np.dot(np.linalg.inv(xtx),xy)
    #e=y-np.dot(tx,w)
    #mse=np.dot(e.transpose(),e)/(2*len(tx))
    return w
    # ***************************************************

In [ ]:
def ridge_regression(y, tx, lamb):
    """implement ridge regression."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # ridge regression: TODO
    xtx=np.dot(tx.transpose(),tx)
    l_inside=2*lamb*len(tx)*np.eye(tx.shape[1])
    ins=xtx+l_inside
    xy=np.dot(tx.transpose(),y)
    
    w=np.dot(np.linalg.inv(ins),xy)
    e=y-np.dot(tx,w)
    mse=np.dot(e.transpose(),e)/(2*len(tx))
    
    return w
    # ***************************************************
    raise NotImplementedError

In [ ]:
def compute_loss(y, tx, w):
    """Calculate the loss.
    
    You can calculate the loss using mse or mae.
    """
    # ***************************************************
    e=y-np.dot(tx,w)
    mse=np.dot(e.transpose(),e)/(2*len(tx))
    return mse
    # ***************************************************
    raise NotImplementedError

In [ ]:
def compute_gradient(y, tx, w):
    """Compute the gradient."""
    # ***************************************************
    e=y-np.dot(tx,w)
    return (-1/len(tx))*np.dot(tx.transpose(),e)
    # ***************************************************
    raise NotImplementedError

In [ ]:
def least_squares_GD(y, tx, gamma, max_iters):
    """Gradient descent algorithm."""
    # Define parameters to store w and loss
    initial_w=np.zeros(tx.shape[1])
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        # ***************************************************
        # INSERT YOUR CODE HERE
        # TODO: compute gradient and loss
        grad=compute_gradient(y,tx,w)
        loss=compute_loss(y,tx,w)
        # ***************************************************
        gamma=gamma/1.005
        # ***************************************************
        # INSERT YOUR CODE HERE
        # TODO: update w by gradient
        w=w-gamma*grad
        # ***************************************************
        
        # store w and loss
        ws.append(np.copy(w))
        losses.append(loss)
        print("Gradient Descent({bi}/{ti}): loss={l}, w0={w0}, w1={w1}".format(
              bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]))

    return losses, ws

In [ ]:
from plots import gradient_descent_visualization
import datetime
# Define the parameters of the algorithm.
max_iters = 100
gamma = 0.00001

# Initialization
#w_initial = np.array([0.0, 0.0])

# Start gradient descent.
start_time = datetime.datetime.now()
gradient_losses, gradient_ws = least_squares_GD(y, tx, gamma, max_iters)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("Gradient Descent: execution time={t:.3f} seconds".format(t=exection_time))

In [ ]:
least_squares_w=least_squares(y,tx)

In [ ]:
lamb=0.00075
ridge_w=ridge_regression(y, tx, lamb)

In [ ]:
testx=np.delete(testx,[14,15,17,18,20,24,25,27,28],1)
testx[testx==-999]=0
m2=np.mean(testx,axis=0)
centered_testx=testx-m2
centered_testx[centered_testx==-m2]=0
standardized_testx=centered_testx / np.std(centered_testx, axis=0)


d=len(standardized_testx[0])
n=len(standardized_testx)
mat=np.zeros((n,1771+int(d*(d+1)/2+d)))

q=0
tao=d 

for i in range (n):
    for t in range (d):  
        mat[i,t]=standardized_testx[i,t]

        
for i in range (n):
    for monsoon in range (d):
        for t in range (monsoon,d): 
            mat[i,tao]=standardized_testx[i,t]*standardized_testx[i,monsoon]
            tao=tao+1
    tao=d  
    
    

tao=int(d*(d+1)/2+d)
    
for i in range (n):
    for r in range (d):
        for monsoon in range (r,d):
            for t in range (monsoon,d): 
                mat[i,tao]=standardized_testx[i,t]*standardized_testx[i,monsoon]*standardized_testx[i,r]
                tao=tao+1
    tao=int(d*(d+1)/2+d)  

    
num_samples=len(standardized_testx)
final_testx = np.c_[np.ones(num_samples), mat]

In [ ]:
w=ridge_w
#w=least_squares_w
#w=gradient_ws[-1]
aq=np.dot(final_testx,w)
y_guess=np.sign(aq)
trueness=sum((y_guess==testy))/len(testy)
print(trueness)

In [4]:
DATA_TEST_PATH = '/Users/alperkose/Desktop/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [ ]:
testx=tX_test
testx=np.delete(testx,[14,15,17,18,20,24,25,27,28],1)
testx[testx==-999]=0
m2=np.mean(testx,axis=0)
centered_testx=testx-m2
centered_testx[centered_testx==-m2]=0
standardized_testx=centered_testx / np.std(centered_testx, axis=0)





d=len(standardized_testx[0])
n=len(standardized_testx)
mat=np.zeros((n,1771+int(d*(d+1)/2+d)))
q=0
tao=d 

for i in range (n):
    for t in range (d):  
        mat[i,t]=standardized_testx[i,t]

        
for i in range (n):
    for monsoon in range (d):
        for t in range (monsoon,d): 
            mat[i,tao]=standardized_testx[i,t]*standardized_testx[i,monsoon]
            tao=tao+1
    tao=d  
    
    

tao=int(d*(d+1)/2+d)
    
for i in range (n):
    for r in range (d):
        for monsoon in range (r,d):
            for t in range (monsoon,d): 
                mat[i,tao]=standardized_testx[i,t]*standardized_testx[i,monsoon]*standardized_testx[i,r]
                tao=tao+1
    tao=int(d*(d+1)/2+d)  


num_samples=len(standardized_testx)
final_testx = np.c_[np.ones(num_samples), mat]
tX_test=final_testx
weights=ridge_w

In [ ]:
OUTPUT_PATH = '/Users/alperkose/Desktop/deneme9.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

In [5]:
sum(tX_test==-999)

array([ 86488,      0,      0,      0, 402796, 402796, 402796,      0,
            0,      0,      0,      0, 402796,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0, 227458,
       227458, 227458, 402796, 402796, 402796,      0])